In [4]:
%pip install gcsfs

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install fsspec

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

In [2]:
df_review = pd.read_parquet('gs://data_cruda/Yelp/review.parquet')

In [3]:
df_business = pd.read_parquet('gs://data_limpia/Yelp/business.parquet')
df_business.shape

(24645, 7)

In [4]:
fecha_desde = pd.to_datetime('2018-01-01')
fecha_hasta = pd.to_datetime('2021-12-31')

# Filtra las filas que están dentro del rango de fechas [fecha_desde, fecha_hasta]
df_review = df_review[(df_review['date'] >= fecha_desde) & (df_review['date'] <= fecha_hasta)]

# Crea df_carga con las filas que están después de la fecha_hasta
df_carga = df_review[df_review['date'] > fecha_hasta]


In [5]:
#Filtramos el df 'review'
review_filtrado = df_review[df_review["business_id"].isin(df_business["business_id"])]
review_filtrado = review_filtrado.merge(df_business[["business_id", "state"]], on="business_id", how="inner")
review_filtrado.shape

(886259, 10)

In [6]:
review_filtrado.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,state
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,PA
1,uyS0ysaMd4mzw5rNYbgcjA,ql0XsKTjM7VeBAUqbphQDw,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"Food is fantastic, service is quite awful! Ca...",2018-03-24 17:50:37,PA
2,R10wk4xEHX9r-qs5Z_2vvw,ZeBgfIMxp9K8OFmlXmQ3yA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,Update: I deducted a star because they no long...,2018-07-21 09:26:33,PA
3,pDN3hRBarmGWXbK64A83MA,IBrReMAeZkVIbjZIe1E_Hw,XQfwVwDr-v0ZS3_CbbE5Xw,1,0,0,0,never coming back here again. all of the glass...,2018-09-08 17:03:53,PA
4,HxWtq5q4OQ-4osStqn54bA,k4_8Cw2icH0nFV5MskGK1A,XQfwVwDr-v0ZS3_CbbE5Xw,2,0,0,0,Unfortunately the weekend chef doesn't know ho...,2018-09-09 14:30:29,PA


In [7]:
review_filtrado = review_filtrado.reindex()
review_filtrado.shape

(886259, 10)

In [8]:
review_filtrado.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date', 'state'],
      dtype='object')

# Creamos la tabla dates

In [9]:
unique_dates = review_filtrado['date'].dt.date.unique()

# Creamos un DataFrame con las fechas únicas
dates = pd.DataFrame({'date': unique_dates})
dates = dates.sort_values(by='date')

dates = dates.reset_index(drop=True).reset_index().rename(columns={'index': 'date_id'})

dates.head(10)

,date_id,date
0,0,2018-01-01
1,1,2018-01-02
2,2,2018-01-03
3,3,2018-01-04
4,4,2018-01-05
5,5,2018-01-06
6,6,2018-01-07
7,7,2018-01-08
8,8,2018-01-09
9,9,2018-01-10


In [10]:
# Reemplazamos los valores de la columna "date" en review_df por los IDs correspondientes
date_to_id = dates.set_index('date')['date_id'].to_dict()

review_filtrado['date'] = review_filtrado['date'].dt.date.map(date_to_id)

review_filtrado.rename(columns={'date': 'date_id'}, inplace = True)

review_filtrado.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date_id,state
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",187,PA
1,uyS0ysaMd4mzw5rNYbgcjA,ql0XsKTjM7VeBAUqbphQDw,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"Food is fantastic, service is quite awful! Ca...",82,PA
2,R10wk4xEHX9r-qs5Z_2vvw,ZeBgfIMxp9K8OFmlXmQ3yA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,Update: I deducted a star because they no long...,201,PA
3,pDN3hRBarmGWXbK64A83MA,IBrReMAeZkVIbjZIe1E_Hw,XQfwVwDr-v0ZS3_CbbE5Xw,1,0,0,0,never coming back here again. all of the glass...,250,PA
4,HxWtq5q4OQ-4osStqn54bA,k4_8Cw2icH0nFV5MskGK1A,XQfwVwDr-v0ZS3_CbbE5Xw,2,0,0,0,Unfortunately the weekend chef doesn't know ho...,251,PA


In [11]:
# Guardar como archivo Parquet
parquet_file_path = "gs://data_limpia/Yelp/dates.parquet"
dates.to_parquet(parquet_file_path, index=False)

In [12]:
# Guardar como archivo Parquet
parquet_file_path = "gs://data_limpia/Yelp/review.parquet"
review_filtrado.to_parquet(parquet_file_path, index=False)

# Trabajamos con el dataset User

In [13]:
parquet_file_path = "gs://data_cruda/Yelp/user.parquet"

user = pd.read_parquet(parquet_file_path)
user.shape

(2105597, 22)

In [14]:
user.drop_duplicates(inplace=True)

In [15]:
user_filtrado = user[user["user_id"].isin(review_filtrado["user_id"])]
user_filtrado.shape

(354508, 22)

In [16]:
user_filtrado = user_filtrado.reindex()
user_filtrado.columns

Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')

In [17]:
#Eliminamos columnas que no se usarán
user_filtrado.drop(columns=[ 'useful', 'funny', 'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot', \
       'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain',\
       'compliment_cool', 'compliment_funny', 'compliment_writer', 'compliment_photos'], inplace=True)

In [18]:
user_filtrado.columns

Index(['user_id', 'name', 'review_count', 'yelping_since'], dtype='object')

In [19]:
user_filtrado.fillna('', inplace=True)

In [20]:
# Guardar como archivo Parquet
parquet_file_path = "gs://data_limpia/Yelp/user.parquet"
user_filtrado.to_parquet(parquet_file_path, index=False)

# Trabajamos con el dataset Tip

In [21]:
# URL del archivo parquet en Google Cloud Storage
gcs_url = 'gs://data_cruda/Yelp/tip.json'

# Leer el archivo parquet en un DataFrame de Pandas
df_tip = pd.read_json(gcs_url, lines=True)

In [22]:
df_tip.shape

(908915, 5)

In [23]:
df_tip.drop_duplicates(inplace=True)

In [24]:
tip_filtradoBus = df_tip[df_tip["business_id"].isin(df_business["business_id"])]
tip_filtradoBus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293031 entries, 1 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           293031 non-null  object        
 1   business_id       293031 non-null  object        
 2   text              293031 non-null  object        
 3   date              293031 non-null  datetime64[ns]
 4   compliment_count  293031 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 13.4+ MB


In [25]:
tip_filtradoUser = tip_filtradoBus[tip_filtradoBus["user_id"].isin(user_filtrado["user_id"])]
tip_filtradoUser.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157793 entries, 3 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           157793 non-null  object        
 1   business_id       157793 non-null  object        
 2   text              157793 non-null  object        
 3   date              157793 non-null  datetime64[ns]
 4   compliment_count  157793 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 7.2+ MB


In [26]:
# Identificar duplicados en el DataFrame
duplicates = tip_filtradoUser.duplicated(subset=['user_id', 'business_id', 'date'])

# Filtrar el DataFrame original para mostrar los duplicados
duplicated_rows = tip_filtradoUser[duplicates]

print(duplicated_rows)

                       user_id             business_id  \
405533  TkFfkUhNLczx5LB-uR9qaw  7EhTT4iEuA7JaaWA-eI3Qw   
409403  TkFfkUhNLczx5LB-uR9qaw  hazg2m7sdLhP2sUD4OPLmw   
592272  Mv6xVzDKY6fA2Ph4etpExQ  KXecZki-_3N7afs6HKWM3A   
688539  xfOK3q6Vsj-7K7qZmby9ew  VAIOtbsxdNmuhUKkQZFRlA   

                                                     text                date  \
405533  Seasons will close Tuesday July 26th and reope... 2016-07-24 04:26:58   
409403  Catering is available. Make waves at your next... 2016-07-11 13:16:17   
592272  Don't come here expecting to do a to go order!... 2016-07-12 06:15:43   
688539                             Get the croque madame! 2011-06-25 20:08:20   

        compliment_count  
405533                 0  
409403                 0  
592272                 0  
688539                 0  


In [27]:
tip_filtradoUser = tip_filtradoUser.reindex()
tip_filtradoUser.drop_duplicates(subset=['user_id', 'business_id', 'date'], inplace=True)

In [28]:
tip_filtradoUser.columns
#Eliminamos columnas que no se usarán
tip_filtradoUser.drop(columns=['compliment_count'], inplace=True)

In [29]:
# Guardar como archivo Parquet
parquet_file_path = "gs://data_limpia/Yelp/tip.parquet"
tip_filtradoUser.to_parquet(parquet_file_path, index=False)